# Set Up Env

In [1]:
import sys
import os

import sagemaker
from sagemaker import get_execution_role

# Add the parent directory to the sys.path
sys.path.insert(0, os.path.abspath('..'))

# Define IAM role
role = get_execution_role()
role

# Establish S3 bucket connection
import boto3
s3 = boto3.client('s3')
bucket = 'capstone-bucket-4-friends'

print(os.getcwd())

from file_utilities import s3_download 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
/home/sagemaker-user/capstone-2024-summer/src/jenna


In [2]:
# standard libraries
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
!pip install seaborn -q
import seaborn as sns

In [3]:
#crsp_and_comp = "CRSP/sp500_crsp_compustat_merged_2018_2023.csv"
#crsp_and_comp_path = s3_download(crsp_and_comp)

In [4]:
#index = "Index/security_master.csv"
#index_path = s3_download(index)

In [12]:
crsp_and_comp_df = pd.read_csv("/home/sagemaker-user/capstone-2024-summer/data/sp500_crsp_compustat_merged_2018_2023.csv")

crsp_and_comp_df.head()

/tmp/ipykernel_968/3900309308.py:1: DtypeWarning: Columns (0,21,25,26,27,28,29,30,31,32,33,37,38,39,40,43,44,45,46,47,52,53,54,55,657,658) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_and_comp_df = pd.read_csv("/home/sagemaker-user/capstone-2024-summer/data/sp500_crsp_compustat_merged_2018_2023.csv")


,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,costat,fic,cshtrq,dvpspq,dvpsxq,mkvaltq,prccq,prchq,prclq,adjex
0,68389X10,10104,8045,10536,1,7379,2018-01-02,46.170,47.8011,46.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68389X10,10104,8045,10536,1,7379,2018-01-03,47.440,48.0700,47.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68389X10,10104,8045,10536,1,7379,2018-01-04,47.715,48.1900,48.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,68389X10,10104,8045,10536,1,7379,2018-01-05,48.280,48.6300,48.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68389X10,10104,8045,10536,1,7379,2018-01-08,47.940,49.0700,48.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# crsp_and_comp_df.columns.tolist()

In [8]:
null_table = crsp_and_comp_df.isnull().sum().reset_index()
null_table.columns = ["Column", "Null Values"]

# Add column to show the percentage of null values
null_table["Percentage"] = (null_table["Null Values"] / len(crsp_and_comp_df)) * 100

# Sorting the table by the number of null values in descending order
null_table_sorted = null_table.sort_values(by="Null Values", ascending=False).reset_index(drop=True)
null_table_sorted

,Column,Null Values,Percentage
0,ufretsdy,828593,100.0
1,xopty,828593,100.0
2,xoptqpy,828593,100.0
3,xoptepsy,828593,100.0
4,xoptepsqpy,828593,100.0
...,...,...,...
662,permno,0,0.0
663,hsiccd,0,0.0
664,hexcd,0,0.0
665,permco,0,0.0


In [14]:
og_cols = len(crsp_and_comp_df.columns)
columns_to_drop = null_percentages[null_percentages > 80].index
crsp_and_comp_df = crsp_and_comp_df.drop(columns=columns_to_drop)
new_cols = len(crsp_and_comp_df.columns)
print("num_columns_removed: ", og_cols - new_cols)
print("num_columns_kept:", new_cols)

num_columns_removed:  310
num_columns_kept: 357


In [10]:
crsp_and_comp_df.head()

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,costat,fic,cshtrq,dvpspq,dvpsxq,mkvaltq,prccq,prchq,prclq,adjex
0,68389X10,10104,8045,10536,1,7379,2018-01-02,46.170,47.8011,46.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68389X10,10104,8045,10536,1,7379,2018-01-03,47.440,48.0700,47.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68389X10,10104,8045,10536,1,7379,2018-01-04,47.715,48.1900,48.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,68389X10,10104,8045,10536,1,7379,2018-01-05,48.280,48.6300,48.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68389X10,10104,8045,10536,1,7379,2018-01-08,47.940,49.0700,48.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def date_formatting(df):
    """Convert to datetime"""
    date_columns = [
        "nameendt",
        "dclrdt",
        "dlpdt",
        "nextdt",
        "paydt",
        "rcrddt",
        "shrenddt",
        "datadate",
        "apdedateq",
        "fdateq",
        "pdateq",
        "rdq",
        "adjex",
        "ipodate",
    ]
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")

date_formatting(crsp_and_comp_df)